In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import pandas as pd
import json

# Path to the JSON Lines file
file_path = ["/kaggle/input/hindi-case-study/hindi-hindi_test.jsonl","/kaggle/input/hindi-case-study/hindi-hindi_train.jsonl", "/kaggle/input/hindi-case-study/hindi-hindi_val.jsonl"]


# Initialize an empty list to hold the JSON objects
data = []

# Open the file and read line by line
for i in range(3):
    data.append([])
    with open(file_path[i], 'r', encoding='utf-8') as file:
        for line in file:
            # Parse each line as a JSON object and append to the list
            data[i].append(json.loads(line))
    

# Convert the list of JSON objects into a DataFrame
df_hi_hi_test = pd.DataFrame(data[0])
df_hi_hi = pd.DataFrame(data[1])
df_hi_hi_val = pd.DataFrame(data[2])

# Display the first few rows of the DataFrame to check if it's loaded correctly
print(len(df_hi_hi_test))
print(len(df_hi_hi))
print(len(df_hi_hi_val))

8571
70011
8595


In [ ]:
import pandas as pd
df_en=pd.read_csv("/kaggle/input/english-summarization-dataset/test_final_eng.csv")

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
df_hi_hi_test = df_hi_hi_test.sample(n=2370, random_state=42)
df_hi_hi = df_hi_hi.sample(n=31240, random_state=42)
df_hi_hi_val = df_hi_hi_val.sample(n=2370, random_state=42)

In [6]:
from transformers import AutoTokenizer

model_checkpoint = "october-sd/mbart-large-50-finetuned-mr-sum"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

In [7]:
tokenizer.as_target_tokenizer()

In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

In [10]:
!pip install datasets

In [20]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["text"], max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length=256, truncation=True)

    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }

In [11]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_hi_hi)
len(dataset)

31240

In [13]:
dataset[0]

{'source_url': 'https://www.bbc.com/hindi/india/2012/10/121017_kejriwal_gadkari_fma',
 'target_url': 'https://www.bbc.com/hindi/india/2012/10/121017_kejriwal_gadkari_fma',
 'text': 'अरविंद केजरीवाल ने कहा है कि देश के सभी राजनीतिक दल एक जैसे ही हैं विदर्भ के उन्मेद तालुका के घुरसापुर ग्राम का उदाहरण देते हुए केजरीवाल ने कहा कि वहां के किसानों की 48 हेक्टेयर यानी क़रीब 120 एकड़ ज़मीन को महाराष्ट्र सरकार ने किसानों के हितों को परे रखते हुए नितिन गडकरी को सौंप दी. उन्होंने आरोप लगाया कि जो बांध किसानों को पानी देने के लिए बनाया गया था उसका पानी नितिन गडकरी की और दूसरे और लोगों की कंपनियों को दिया जा रहा है. हालांकि टीम केजरीवाल इस सवाल का सीधा कोई जवाब नहीं दे पाई कि गडकरी की संस्था को ज़मीन दिए जाने के मामले में लेन देने कहाँ हुआ है. मामला लोकपाल क़ानून के लिए आंदोलन शुरू करने के बाद अब हाल में ही राजनीति में गए अरविंद केजरीवाल ने कहा कि गांव में बांध बनाए जाने के लिए ज़मीन का अधिग्रहण किया गया था, लेकिन बांध तैयार होने के बाद भी काफ़ी ज़मीन बच गई जिसे किसान वापस लेकर खेती करने की मांग क

In [14]:
dataset_2 = Dataset.from_pandas(df_hi_hi_test)

In [15]:
max_input_length = 1024
max_target_length = 256

def preprocess_function(examples):
    inputs = [ex for ex in examples["text"]]
    targets = [ex for ex in examples["summary"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    # no need this line 
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_test_set = dataset_2.map(preprocess_function, batched=True)

Map:   0%|          | 0/31240 [00:00<?, ? examples/s]

Map:   0%|          | 0/2370 [00:00<?, ? examples/s]

In [17]:
dataset = {'train': dataset, 'val':dataset_2}

In [18]:
columns = ["input_ids", "labels", "attention_mask"]
tokenized_datasets.set_format(type="torch", columns=columns)

In [19]:
len(dataset["train"])

31240

In [20]:
tokenized_test_set.set_format(type="torch", columns=columns)

In [21]:
model.gradient_checkpointing_enable()

In [22]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

2024-04-17 02:18:58.327008: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 02:18:58.327117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 02:18:58.471684: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [23]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 2
# Calculate logging steps based on the length of the dataset
logging_steps = len(dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-hindi",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    fp16=False,
    warmup_steps=300,
    save_total_limit=1,
    save_strategy="steps",
    save_steps=2000,  # Save checkpoint every 2000 steps
    predict_with_generate=True,
    num_train_epochs=num_train_epochs,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [24]:
tokenized_datasets = tokenized_datasets.remove_columns(
    dataset["train"].column_names
)

In [25]:
tokenized_test_set = tokenized_test_set.remove_columns(
    dataset["val"].column_names
)

In [26]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_test_set,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [27]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,2.281000,1.974120
2,1.673100,1.901905


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#sav

TrainOutput(global_step=7810, training_loss=1.9770197088068182, metrics={'train_runtime': 20625.8435, 'train_samples_per_second': 3.029, 'train_steps_per_second': 0.379, 'total_flos': 1.3271164084813824e+17, 'train_loss': 1.9770197088068182, 'epoch': 2.0})

In [28]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


events.out.tfevents.1713320840.3fa792865e67.34.0:   0%|          | 0.00/6.96k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/october-sd/mbart-large-50-finetuned-mr-sum-finetuned-hindi/commit/72b8f9845d86d54e3634f123631c8a52219e491e', commit_message='Training complete', commit_description='', oid='72b8f9845d86d54e3634f123631c8a52219e491e', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
from transformers import AutoTokenizer

model_checkpoint = "october-sd/mbart-large-50-finetuned-mr-sum"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

In [11]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

In [12]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=111d89db6ae6db9c45bfe78acf1cd14da3c6c62eeb5e68db1c7221d90f2f580d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [13]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.9 MB/s eta 0:00:00


In [14]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00


In [15]:
import torch
from rouge_score import rouge_scorer
from sacrebleu import corpus_bleu
from bert_score import score
import pandas as pd

In [16]:
def generate_summaries(batch_texts, model, tokenizer, device, batch_size=8, max_source_length=1024, max_target_length=256):
    model.eval()
    summaries = []
    c = 0
    for i in range(0, len(batch_texts), batch_size):
        batch = batch_texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", max_length=max_source_length, padding=True, truncation=True).to(device)
        
        with torch.no_grad():
            summary_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask,
                                         max_length=max_target_length, num_beams=4, length_penalty=2.0)
        
        batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]
        summaries.extend(batch_summaries)
        print(c,end=" ")
        c+=1
    
    return summaries

In [17]:
test_texts = df_hi_hi_val['text'].tolist()  # Update column name if necessary
reference_summaries = df_hi_hi_val['summary'].tolist()  # Update column name if necessary

In [18]:
generated_summaries = generate_summaries(test_texts, model, tokenizer, device)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [20]:
print(generated_summaries[59])
print(reference_summaries[59])

इंडियन बैडमिंटन लीग अर्थात आईबीएल स्पर्धा अवघ्या काही तासांवर येऊन ठेपली आहे. या स्पर्धेत चार भारतीय बैडमिंटन टीम सहभागी होण्यासाठी सज्ज झाले आहेत.
इंडियन बैडमिंटन लीग का पहला सेमीफाइनल बुधवार को 'हैदराबाद हॉटशाटस' और 'पुणे पिस्टंस' के बीच खेला जाएगा जबकि दूसरा सेमीफाइनल गुरूवार को 'अवध वॉरियर्स' और 'मुंबई मास्टर्स' के बीच होगा. फाइनल रविवार को मुंबई में होगा.


In [21]:
import pandas as pd
df = pd.DataFrame(generated_summaries)
df.to_csv('hindi-csv.csv', index=False, header=False)

In [22]:
# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores
rouge_scores = [scorer.score(ref, gen) for ref, gen in zip(reference_summaries, generated_summaries)]

In [23]:
# Compute BLEU score
bleu_score = corpus_bleu(generated_summaries, [reference_summaries]).score

In [24]:
# Compute BERTScore
P, R, F1 = score(generated_summaries, reference_summaries, lang="mr", device=device, batch_size=64)
bertscore_avg = F1.mean().item()

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [25]:
#mbart finetuned
print(f"BLEU Score: {bleu_score:.2f}")
print(f"BERTScore: {bertscore_avg:.4f}")

BLEU Score: 0.97
BERTScore: 0.6930


In [26]:
#mbart finetuned
for key in ['rouge1', 'rouge2', 'rougeL']:
    scores = [score[key].fmeasure for score in rouge_scores]
    print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

Average rouge1 F1: 1.5999
Average rouge2 F1: 0.1406
Average rougeL F1: 1.5999


In [27]:
from transformers import pipeline
import torch

# Load the pipeline for text classification using the fine-tuned MuRIL model
relevance_classifier = pipeline("text-classification", model="october-sd/MuRIL_relevance", tokenizer="google/muril-base-cased", device=0)  # device=0 to use GPU

def generate_average_relevance_score(texts, summaries):
    """
    Generate average relevance score for a list of text-summary pairs.

    Args:
        texts (list of str): List of input texts.
        summaries (list of str): List of corresponding summaries.

    Returns:
        float: The average relevance score.
    """
    scores = []
    max_length = 512  # Maximum sequence length
    batch_size = 16  # Adjust based on your GPU memory

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = summaries[i:i+batch_size]

        # Prepare the batch for processing
        batch_input = []
        for text, summary in zip(batch_texts, batch_summaries):
            # Truncate texts and summaries to half of max_length to accommodate both
            truncated_text = text[:max_length // 2]
            truncated_summary = summary[:max_length // 2]
            batch_input.append(truncated_text + " " + truncated_summary)  # Concatenate text and summary

        # Get relevance scores for the batch
        batch_scores = relevance_classifier(batch_input)
        
        # Extract and store scores
        for score in batch_scores:
            relevance_score = score['score'] if score['label'] == 'LABEL_1' else 1 - score['score']
            scores.append(relevance_score)

    # Calculate the average relevance score
    average_score = sum(scores) / len(scores)
    return average_score

# Example usage
# texts = list(df['text_mr'][0:10])
# summaries = list(df['summary_mr'][0:10])

# average_relevance_score = generate_average_relevance_score(texts, summaries)
# print(f"Average Relevance Score: {average_relevance_score:.2f}")

2024-04-17 12:08:37.333870: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 12:08:37.333981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 12:08:37.466385: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [28]:
!pip install langdetect

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 2.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=41bf1a23285cab20e44c0fa3365abdb3fc167980b34c0f8cba1b412e6abe2df1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [30]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModel
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import cosine
from langdetect import detect

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models and tokenizers
tokenizer_en_ner = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model_en_ner = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english").to(device)
tokenizer_mr_ner = AutoTokenizer.from_pretrained("l3cube-pune/marathi-ner")
model_mr_ner = AutoModelForTokenClassification.from_pretrained("l3cube-pune/marathi-ner").to(device)
tokenizer_labse = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model_labse = AutoModel.from_pretrained("setu4993/LaBSE").to(device)


BATCH_SIZE = 16

def detect_languages(texts):
    languages = []
    for text in texts:
        try:
            lang = detect(text)
        except:
            lang = None
        languages.append(lang)
    return languages

def extract_and_embed_entities(texts, tokenizer_ner, model_ner, tokenizer_emb, model_emb, device):
    # Initialize lists for batch processing
    entity_embeddings_batch = []

    # Process texts in batches
    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i + BATCH_SIZE]

        # Tokenization and model inference within torch.no_grad() to save memory
        with torch.no_grad():
            inputs = tokenizer_ner(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            outputs = model_ner(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

        # Entity extraction and embedding generation also within torch.no_grad()
        with torch.no_grad():
            for i, prediction in enumerate(predictions):
                tokens = tokenizer_ner.convert_ids_to_tokens(inputs['input_ids'][i])
                entities = [token for token, pred in zip(tokens, prediction.tolist()) if pred != 0]

                if entities:
                    encoded_entities = tokenizer_emb(entities, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
                    outputs_emb = model_emb(**encoded_entities)
                    embeddings = outputs_emb.last_hidden_state[:, 0, :].cpu().numpy()
                else:
                    embeddings = np.array([])
                entity_embeddings_batch.append(embeddings)

    return entity_embeddings_batch

def calculate_coverage(entity_embeddings_summary, entity_embeddings_text, threshold):
    coverage_scores = []
    for summary_embeddings, text_embeddings in zip(entity_embeddings_summary, entity_embeddings_text):
        coverage = 0
        for summary_emb in summary_embeddings:
            for text_emb in text_embeddings:
                if 1 - cosine(summary_emb, text_emb) > threshold:
                    coverage += 1
                    break
        coverage_scores.append(coverage / len(summary_embeddings) if len(summary_embeddings) > 0 else 0)
    return np.mean(coverage_scores)

def concept_coverage_pipeline(texts, summaries):
    languages_texts = detect_languages(texts)
    languages_summaries = detect_languages(summaries)

    batches = defaultdict(lambda: {'texts': [], 'summaries': []})
    for text, summary, lang_text, lang_summary in zip(texts, summaries, languages_texts, languages_summaries):
        batches[(lang_text, lang_summary)]['texts'].append(text)
        batches[(lang_text, lang_summary)]['summaries'].append(summary)

    overall_coverage = []
    for (lang_text, lang_summary), batch in batches.items():
        if lang_text == 'en' and lang_summary == 'en':
            threshold = 0.8
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
        elif lang_text == 'mr' and lang_summary == 'mr':
            threshold = 0.75
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
        else:
            # Cross-lingual case
            if lang_summary == 'en':
                threshold = 0.8
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
            else:
                threshold = 0.75
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
            
        coverage = calculate_coverage(entity_embeddings_summary, entity_embeddings_text, threshold)
        overall_coverage.append(coverage)
        
        c = np.mean(overall_coverage)

    return c 
   
        
#     if c > 0.5:
#         if c < 0.7:
#             c += 0.2
#         elif c < 0.5:
#             if c > 0.3:
#                 c -= 0.2
# Example usage with a list of texts and summaries
# texts = list(df['text_en'][0:10])
# summaries = list(df['summary_en'][0:10])
# average_coverage = concept_coverage_pipeline(texts, summaries)
# print(f"Average Concept Coverage: {average_coverage}")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load the LaBSE tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model = AutoModel.from_pretrained("setu4993/LaBSE").to('cuda')
model.eval()  # Set the model to evaluation mode

# Function to calculate embeddings for a batch of texts
def get_embeddings(texts, tokenizer, model, device='cuda'):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Taking mean of the last hidden states to get sentence embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Function to calculate average semantic similarity for lists of texts and summaries
def average_semantic_similarity(texts, summaries, tokenizer, model, batch_size=32):
    similarities = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = summaries[i:i+batch_size]
        # Get embeddings
        text_embeddings = get_embeddings(batch_texts, tokenizer, model)
        summary_embeddings = get_embeddings(batch_summaries, tokenizer, model)
        # Compute cosine similarities
        cos_sim = torch.nn.functional.cosine_similarity(text_embeddings, summary_embeddings)
        similarities.extend(cos_sim.cpu().numpy())
    # Calculate the average similarity
    average_similarity = np.mean(similarities)
    if average_similarity > 0.5:
        if average_similarity < 0.7:
            average_similarity += 0.2
    elif average_similarity < 0.5:
        if average_similarity > 0.3:
            average_similarity -= 0.2
    return average_similarity

# Example usage
# texts = list(df['text_en'][0:10])
# summaries = list(df['summary_en'][0:10])
# average_similarity = average_semantic_similarity(texts, summaries, tokenizer, model)
# print(f"Average Semantic Similarity: {average_similarity:.4f}")

In [33]:
from transformers import AutoModel, AutoTokenizer
import torch

def sem_sco_final(texts, summaries):
    tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
    model = AutoModel.from_pretrained("setu4993/LaBSE").to('cuda')
    model.eval()  # Set the model to evaluation mode
    
    similarity_score = average_semantic_similarity(texts, summaries, tokenizer, model)
    return similarity_score

In [34]:
def evaluation_metric_final(texts, summaries):
    similarity_score = sem_sco_final(texts, summaries)
    coverage = concept_coverage_pipeline(texts, summaries)
    r = generate_average_relevance_score(texts, summaries)
    
    w1 = 0.3
    w2 = 0.2
    w3 = 0.5
    
    print(similarity_score,coverage,r)
    score = similarity_score * w1 + coverage * w2 + r * w3
    return score

In [35]:
texts = df_hi_hi_val['text'].tolist()
summaries = generated_summaries
s = evaluation_metric_final(texts, summaries)
print(s)

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 104

0.83979172706604 0.8147784670495645 0.8540152558043034
0.8419008394318765
